# Dataset preparation

In this notebook we clean our dataset from deleted users, inactive users and users with incorrect personality flag.

In [117]:
# Import libreries.
import pandas as pd
import numpy as np

In [118]:
# Import dataframes.
df_posts = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/total_post_2.csv", dtype={'id':str})
df_comments = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/total_comm_2.csv", dtype={'id':str})

C:\Users\HP\AppData\Local\Temp\ipykernel_6644\4178844003.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posts = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/total_post_2.csv", dtype={'id':str})


In [119]:
# Every id must be a string.
df_comments['id'] = df_comments['id'].astype(str)
df_posts['id'] = df_posts['id'].astype(str)

In [120]:
# Remove errors in id.
def check_id(x):
    if x.startswith('t3_') or x.startswith('t1_'):
        return x[:3]
    else: return x
df_comments['id'] = df_comments['id'].apply(check_id)
df_posts['id'] = df_posts['id'].apply(check_id)

In [121]:
# Drop all Nan values.
df_comments = df_comments.dropna(subset=['link_id']).copy()

In [122]:
# Convert created_utc columns to datetime
df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'], unit='s')
df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'], unit='s' , errors='coerce')
df_comments['year'] = df_comments['created_utc'].apply(lambda x: str(x)[:4])
df_comments = df_comments[df_comments['year'].isin(['2020','2021','2022'])]
df_posts['year'] = df_posts['created_utc'].apply(lambda x: str(x)[:4])
df_posts = df_posts[df_posts['year'].isin(['2020','2021','2022'])]

combined_df = pd.concat([df_comments[['author', 'created_utc']], df_posts[['author', 'created_utc']]])

# Group by author and calculate the first and last activity date
user_activity = combined_df.groupby('author')['created_utc'].agg(['min', 'max'])

# Calculate the duration of activity in days
user_activity['duration'] = (user_activity['max'] - user_activity['min']).dt.days

# Filter users active for at least 14 days
active_users = list(user_activity[user_activity['duration'] >= 14].index)

C:\Users\HP\AppData\Local\Temp\ipykernel_6644\2659431956.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'], unit='s' , errors='coerce')


In [123]:
flair_df = pd.concat([df_comments[['author', 'author_flair_css_class']], df_posts[['author', 'author_flair_css_class']]]).drop_duplicates()

# Keeps users with mtbi flag.
mbti = ['intj', 'intp', 'entj', 'entp', 'infj', 'infp', 'enfj', 'enfp', 'istj', 'isfj', 'estj', 'esfj', 'istp', 
        'isfp', 'estp', 'esfp']
flair_df = flair_df[flair_df['author_flair_css_class'].isin(mbti)]

# Create a list with all duplicated authors.
duplicates = flair_df.duplicated(subset=['author'], keep=False)

# Drop rows where duplicates are found.
flair_df = flair_df[~duplicates]

# List of users with correct flair class.
correct_flair_users = list(flair_df['author'])

### Filters
+ Inactivity filter (through active_users)
+ Flair filter (for every author that has more than 1 flair class/ that has no flair class)
+ Deleted filter (for every deleted author)

In [124]:
# Inactivity filter.
df_comments['inactivity_filter'] = df_comments['author'].isin(active_users)
df_posts['inactivity_filter'] = df_posts['author'].isin(active_users)

In [125]:
# Flair filter.
df_comments['flair_filter'] = df_comments['author'].isin(correct_flair_users)
df_posts['flair_filter'] = df_posts['author'].isin(correct_flair_users)

In [126]:
# Deleted filter.
df_comments['deleted_filter'] = df_comments['author'] != '[deleted]'
df_posts['deleted_filter'] = df_posts['author'] != '[deleted]'

In [127]:
df_comments['filter'] = df_comments['inactivity_filter'] & df_comments['flair_filter'] & df_comments['deleted_filter']
df_posts['filter'] = df_posts['inactivity_filter'] & df_posts['flair_filter'] & df_posts['deleted_filter']

In [128]:
df_comments = df_comments.drop(columns=['Unnamed: 0'])
df_posts = df_posts.drop(columns=['Unnamed: 0'])

Every row with a False in filter column must be neglected. We now need to link the comment chains back in order to mantain the interactions between authors even if the comment chain is interrupted by an invalid id.

In [129]:
df_posts

,author,author_flair_css_class,created_utc,id,year,inactivity_filter,flair_filter,deleted_filter,filter
49882,muddy120,infj,2020-01-01 00:10:40,eib5d8,2020,True,True,True,True
49883,prometheus_x,NaN,2020-01-01 00:32:00,eibeol,2020,True,True,True,True
49884,Rolantis,intp,2020-01-01 01:04:00,eibseu,2020,True,True,True,True
49885,silver_starfire,infp,2020-01-01 01:12:32,eibwgh,2020,True,True,True,True
49886,Comrade_Savva,NaN,2020-01-01 01:14:40,eibx0x,2020,False,False,True,False
...,...,...,...,...,...,...,...,...,...
192589,[deleted],NaN,2022-12-31 23:29:28,1006j8c,2022,True,False,False,False
192590,High__IQ__ENTJ,entj,2022-12-31 23:30:06,1006jo0,2022,False,True,True,False
192591,chaos-seeker320,estj,2022-12-31 23:31:53,1006kz8,2022,False,True,True,False
192592,Ash__Williams,NaN,2022-12-31 23:32:52,1006lmd,2022,True,True,True,True


In [130]:
df_comments

,author_flair_css_class,created_utc,id,author,parent_id,link_id,year,inactivity_filter,flair_filter,deleted_filter,filter
836065,NaN,2020-01-01 00:01:35,fcontgr,[deleted],t3_ei6v13,t3_ei6v13,2020,True,False,False,False
836066,intp,2020-01-01 00:01:37,fcontk0,cornycatlady,t3_ei7x2k,t3_ei7x2k,2020,True,True,True,True
836067,intj,2020-01-01 00:01:42,fconttt,sicsto,t1_fcn6yr0,t3_ehsdhe,2020,True,True,True,True
836068,NaN,2020-01-01 00:03:54,fcoo05f,[deleted],t3_ei7819,t3_ei7819,2020,True,False,False,False
836069,infp,2020-01-01 00:04:01,fcoo0hp,U_Sam,t3_ei7x2k,t3_ei7x2k,2020,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
2961255,intp,2022-12-31 23:54:04,j2fwv5e,Deep_Craft_3760,t3_1003lfy,t3_1003lfy,2022,True,True,True,True
2961256,istp,2022-12-31 23:54:28,j2fwx7a,Zealousideal_Mix_629,t1_j2fsrf2,t3_zzu1je,2022,True,True,True,True
2961257,estj,2022-12-31 23:56:52,j2fx8tx,chaos-seeker320,t1_j2fvunu,t3_1006e8z,2022,False,True,True,False
2961258,NaN,2022-12-31 23:58:42,j2fxhsa,[deleted],t1_j2fwx7a,t3_zzu1je,2022,True,False,False,False


In [131]:
# Get the filtered_out ids.
filtered_out_comm = df_comments[~df_comments['filter']]['id'].tolist()
filtered_out_post = df_posts[~df_posts['filter']]['id'].tolist()
filtered_out = list(set(filtered_out_comm) | set(filtered_out_post))

If the parent_id of a certain comment is inside filtered_out_ids we need to link it to the parent comment of the invalid one.

In [132]:
# df_comments + df_posts.
df_full = pd.concat([df_comments[['id','author','author_flair_css_class']], df_posts[['id','author','author_flair_css_class']]], axis=0)

In [133]:
# Dictionary that links id to author.
id_to_auth = df_full.set_index('id')['author'].to_dict()

In [134]:
# Create parent_comment_id column.
df_comments['parent_comment_id'] = df_comments['parent_id'].apply(lambda x: x[3:])

In [135]:
# Dictionary that links id to parent id.
id_to_parent = df_comments.set_index('id')['parent_comment_id'].to_dict()

In [136]:
# Column with t3 or t1.
df_comments['parent_type'] = df_comments['parent_id'].apply(lambda x: x[:2])

In [137]:
# Divide the datasets in comments of posts and comments of comments.
print(f'Before filter on id = {len(df_comments)}')
df_comments = df_comments[df_comments['filter']]
print(f'After filter on id = {len(df_comments)}')
df_comments = df_comments[~df_comments['parent_comment_id'].isin(filtered_out)]
print(f'After filter on parent_comment_id = {len(df_comments)}')

Before filter on id = 2101833
After filter on id = 1209545
After filter on parent_comment_id = 646919


In [138]:
# Drop all the useless columns.
df_comments = df_comments.drop(columns=['inactivity_filter','flair_filter','deleted_filter'])
df_comments

,author_flair_css_class,created_utc,id,author,parent_id,link_id,year,filter,parent_comment_id,parent_type
836066,intp,2020-01-01 00:01:37,fcontk0,cornycatlady,t3_ei7x2k,t3_ei7x2k,2020,True,ei7x2k,t3
836067,intj,2020-01-01 00:01:42,fconttt,sicsto,t1_fcn6yr0,t3_ehsdhe,2020,True,fcn6yr0,t1
836070,isfp,2020-01-01 00:05:00,fcoo3c7,9741804,t1_fcokncu,t3_ei7x2k,2020,True,fcokncu,t1
836071,intp,2020-01-01 00:05:03,fcoo3gc,mformelancholy,t3_ei7x2k,t3_ei7x2k,2020,True,ei7x2k,t3
836072,entp,2020-01-01 00:05:27,fcoo4pl,SadboiENTrePreneur,t3_ei7dro,t3_ei7dro,2020,True,ei7dro,t3
...,...,...,...,...,...,...,...,...,...,...
2961249,NaN,2022-12-31 23:47:50,j2fw0zb,Zenishira,t1_j2fv4q0,t3_10004bx,2022,True,j2fv4q0,t1
2961250,enfp,2022-12-31 23:47:55,j2fw1go,Icy_Ad6842,t3_zzwz2a,t3_zzwz2a,2022,True,zzwz2a,t3
2961251,intp,2022-12-31 23:48:11,j2fw2qu,Nebris222,t3_1003lfy,t3_1003lfy,2022,True,1003lfy,t3
2961252,estj,2022-12-31 23:48:27,j2fw424,diabolicalfrnchtoast,t1_j2cfdcz,t3_zzlu5n,2022,True,j2cfdcz,t1


In [139]:
# Get all unique ids in posts dataframe.
posts_ids = df_posts.loc[df_posts['filter'],'id'].unique()

In [140]:
# Get all unique ids in comments dataframe.
comments_ids = df_comments.loc[df_comments['filter'],'id'].unique()

In [141]:
# Remove all the rows whit parent_comment_id = NaN.
df_comments = df_comments[~df_comments['parent_comment_id'].isna()]

In [142]:
# Create a columns "parent_author" with the author of the parent comment/submission.
df_comments = df_comments[df_comments['parent_comment_id'].isin(id_to_auth.keys())]
df_comments['parent_author'] = df_comments['parent_comment_id'].apply(lambda x: id_to_auth[x])

In [143]:
# Create a list b in the format needed to build the network.
a = df_comments[['author','parent_author']].values.tolist()

b = []
for elem in a:
    # Don't consider autoloop.
    if elem[0] != elem[1]:
        b.append(f'{elem[0]} {elem[1]}')
        #b.append([elem[0],elem[1]])

In [144]:
# Create a dataframe with edge and its weight.
df_b = pd.DataFrame(b, columns=['edge'])
grouped_df = df_b.groupby(list(df_b.columns)).size().reset_index(name='count')

In [145]:
grouped_df.sort_values('count', ascending=False)

,edge,count
2803,111god7 biwaly,85
97172,Gullible_Performer_6 inefj,73
362547,inefj Gullible_Performer_6,72
305549,biwaly 111god7,67
71094,Either-Fig-9343 Deep_Craft_3760,67
...,...,...
175715,No_Emotion_3849 VarsityQueen,1
175713,No_Emotion_3849 Supreme_Leader_Snob,1
175712,No_Emotion_3849 SoFishtiCating,1
175710,No_Emotion_3849 RareMongoose2602,1


In [146]:
# Create source and target columns.
grouped_df['source'] = grouped_df['edge'].apply(lambda x: x.split(' ')[0])
grouped_df['target'] = grouped_df['edge'].apply(lambda x: x.split(' ')[1])

In [147]:
author_class = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/author_class.csv", usecols=['author','class','i_or_e'])

In [148]:
# Merge to add class to source and target.
merged_df = pd.merge(grouped_df, author_class[['author','class']], left_on='source', right_on='author').drop(columns=['author'])
merged_df.rename(columns={'class':'source_class'}, inplace=True)
merged_df = pd.merge(merged_df, author_class[['author','class']], left_on='target', right_on='author').drop(columns=['author'])
merged_df.rename(columns={'class':'target_class'}, inplace=True)

In [149]:
merged_df

,edge,count,source,target,source_class,target_class
0,--------_-- Juulsteen,1,--------_--,Juulsteen,entp,entp
1,1Zer0Her0 Juulsteen,2,1Zer0Her0,Juulsteen,entp,entp
2,5wings4birds Juulsteen,2,5wings4birds,Juulsteen,intp,entp
3,Ak_707 Juulsteen,1,Ak_707,Juulsteen,intp,entp
4,AlmightyShrimp Juulsteen,2,AlmightyShrimp,Juulsteen,infp,entp
...,...,...,...,...,...,...
466444,zizu232 fancybeekeeper,1,zizu232,fancybeekeeper,entj,entj
466445,znslattt fatafata12,1,znslattt,fatafata12,entp,entj
466446,zoinks27 ravenpvff,1,zoinks27,ravenpvff,infp,infp
466447,zu_capone kukdraws,1,zu_capone,kukdraws,infj,infj


In [150]:
merged_df.to_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/edges.csv")